In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2")


In [3]:
# State
from langgraph.graph import MessagesState

class State(MessagesState):
  summary: str
  

In [4]:
# Node
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage

def conv_node(state):
  # if summary exists: add summary in the beginning as a sys message
  if state.get('summary'):
    messages = [SystemMessage(content=f"Summary of conversation earlier: {state['summary']}")] + state['messages']
    
  else:
    messages = state['messages']
    
  return {
    'messages': llm.invoke(messages)
  }
  
def summary_node(state):
  # Assume the length is already over 5
  # delete messages only leave the last two
  delete_messages = [RemoveMessage(id=m.id) for m in state['messages'][:-2]]
  # summarize all the messages
  
  if state.get('summary'):
    summary_request = f"""
    This is the summary of the previous conversation: {state['summary']}
    \n\n
    Extend the summary with the summary of above conversations
    """
  else:
    summary_request = """
    Create a summary of the above conversations
    """
    
  summary = llm.invoke(state['messages'] + [HumanMessage(content=summary_request)])
  return {
    'summary': summary,
    'messages': delete_messages
  }
  

# edges
def summary_condition(state):
  if len(state['messages']) > 5:
    return 'summary_node'
  else:
    return '__end__'
    
    

In [5]:
from langgraph.graph import StateGraph, START, END
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

builder = StateGraph(State)
builder.add_node(conv_node)
builder.add_node(summary_node)

builder.add_edge(START, 'conv_node')
builder.add_conditional_edges('conv_node', summary_condition)
builder.add_edge('summary_node', END)

db_path = "state_db/workshop.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
memory = SqliteSaver(conn)
graph = builder.compile(checkpointer=memory)

In [14]:
config = {"configurable": {"thread_id": 1}}

msg = "how to I view the data from the db table again?"

state = {
  "messages": [HumanMessage(content=msg)]
}

graph.invoke(state, config)

{'messages': [HumanMessage(content='how to I view the data from the db table again?', additional_kwargs={}, response_metadata={}, id='a005d165-a48a-42ba-bdb1-37807eb061df'),
  AIMessage(content='To view the data from a specific table in an SQLite database using Python sqlite3, you can execute a `SELECT` statement on that table.\n\nHere are some examples:\n\n**Method 1: Viewing all rows**\n\n```python\nimport sqlite3\n\n# Connect to the database\nconn = sqlite3.connect(\'your_database.db\')\ncursor = conn.cursor()\n\n# Execute a SELECT statement to view all rows from a specific table\ntable_name = \'your_table_name\'\ncursor.execute(f"SELECT * FROM {table_name}")\n\n# Fetch all rows from the query result (returns the row data)\nrows_list = cursor.fetchall()\nfor row in rows_list:\n    print(row)\n\n# Close the connection\nconn.close()\n```\n\n**Method 2: Viewing specific columns**\n\n```python\nimport sqlite3\n\n# Connect to the database\nconn = sqlite3.connect(\'your_database.db\')\ncu

In [13]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

table_names_list = cursor.fetchall()

print(table_names_list)

cursor.execute("")

[('checkpoints',), ('writes',)]


In [25]:
cursor.execute("SELECT * FROM checkpoints")

rows_list = cursor.fetchall()
print(rows_list[2])

('1', '', '1ef8febd-fd41-6f54-8001-5b1e99b2eede', '1ef8febd-9f05-6669-8000-aa2726a29a59', 'msgpack', b'\x87\xa1v\x01\xa2ts\xd9 2024-10-21T20:34:30.708206+00:00\xa2id\xd9$1ef8febd-fd41-6f54-8001-5b1e99b2eede\xaechannel_values\x82\xa8messages\x92\xc7\xd8\x05\x94\xbdlangchain_core.messages.human\xacHumanMessage\x87\xa7content\xd9$how do I print the data from the db?\xb1additional_kwargs\x80\xb1response_metadata\x80\xa4type\xa5human\xa4name\xc0\xa2id\xd9$ff264f70-bd2e-4307-a7e1-52ae7c501044\xa7example\xc2\xb3model_validate_json\xc8\n\x9b\x05\x94\xbalangchain_core.messages.ai\xa9AIMessage\x8a\xa7content\xda\x08\x9fTo print data from a database, you\'ll need to use an ORM (Object-Relational Mapping) tool or a direct query method depending on your programming language and the structure of your database. Here\'s a general example using Python with SQLAlchemy (a popular ORM library):\n\n### Installing SQLAlchemy\n\nFirst, install SQLAlchemy if you haven\'t already:\n\n```bash\npip install sqlal